# 栈的顺序表实现

In [1]:
class SStack():
    def __init__(self): # 用list对象_elem存储栈中元素
        self._elems = [] # 所有栈操作都映射到list中
    
    def is_empty(self):
        return self._elems == []

    def top(self):
        if self._elems == []:
            raise StackUnderflow("in SStack.top()")
        return self.elem[-1] # 后进先出，得到最后一个元素
    
    def push(self, elem):
        self._elems.append(elem)
    
    def pop(self):
        if self._elems == []:
            raise StackUnderflow("in SStack.pop()")
        return self._elems.pop() # 以列表形式弹出
    
    def printall(self):
        print(self._elems)

In [2]:
# 使用SStack抽象数据类型
st1 = SStack()
st1.push(3)
st1.push(5)
while not st1.is_empty():
    print(st1.pop())
st1.printall()

5
3
[]


# 栈的链接表实现

In [3]:
# 首先重新定义一下LNode:
class LNode():
    def __init__(self, elem, next_ = None):
        self._elem = elem
        self.next = next_

In [4]:
class LStack(): # 基于链接表技术实现的栈类，用LNode作为结点
    def __init__(self):
        self._top = None
    
    def is_empty(self):
        return self._top is None

    def top(self):
        if self._top is None:
            raise StackUnderflow("in LStack.top()")
        return self._top.elem
    
    def push(self, elem):
        self._top = LNode(elem, self._top)
    
    def pop(self):
        if self._top is None:
            raise StackUnderflow("in LStack pop()")
        p = self._top
        self._top = p.next
        return p.elem
    def printall(self):
        p = self._top
        while p.next is not None:
            print(p)
            p = p.next

In [5]:
#假设list1里存储这要颠倒的列表
list1 = [1, 2, 3, 4, 5]
st1 = SStack()
for x in list1:
    st1.push(x)
list2 = []
while not st1.is_empty():
    list2.append(st1.pop())

list2 # 来看下list2长什么样

[5, 4, 3, 2, 1]

整个操作的代价为O（n）

# 括号匹配问题

In [6]:
def check_parens(text):
    """
    括号匹配检查函数，text是被检查的正文串
    """
    parens = "()[]{}"
    open_parens = "([{"
    opposite = {")":"(", "]": "[", "}": "{"} # 表示配对关系的字典
    
    def parentheses(text):
        """括号生成器，每次调用返回text里的下一括号及其位置"""
        i, text_len = 0, len(text)
        while True:
            while i < text_len and text[i] not in parens:
                i += 1
            if i >= text_len:
                return
            yield text[i], i
            print(text[i])
            i += 1
    
    st = SStack() # 保存括号的栈
    for pr, i in parentheses(text): # 对text里各括号和位置进行迭代
        if pr in open_parens:
            st.push(pr) # 如果是开括号，就押进栈
        elif st.pop() != opposite[pr]: # 如果不匹配就失败
            print("Unmatching is found at", i , "for", pr)
            return False
        else:
            pass # 因为成功了，就啥都不做
        
    print("All parenthese are correctly matched")
#     st.printall()
    return True

# 表达式的变换

In [7]:
# 从SStack继承一个类过来
class ESStack(SStack):
    def __init__(self):
        SStack.__init__(self)
    
    def depth(self): # 增加一个深度表，因为计算的时候需要知道该表的深度
        return len(self._elems)

In [8]:
# 该函数用来分割输入的函数
def suffix_exp_evaluator(line):
    return suf_exp_evaluator(line.split())

In [11]:
# 以下是具体实施细节
def suf_exp_evaluator(exp):
    operators = "+-*/"
    st = ESStack()
    
    for x in exp:
        if x not in operators:
            st.push(float(x)) # 不能转化就会自动引发异常（浮点不得行）
            continue
        
        if st.depth() < 2: #栈元素不够的时候也会引发异常
            raise SystaxError("Short of operand(s).")
        a = st.pop()
        b = st.pop()
        
        if x =="+":
            c = b + a
        elif x == "-":
            c = b - a
        elif x == "*":
            c = b * a
        elif x == "/":
            c = b/a
        else:
            break
        
        st.push(c)
        
    if st.depth() == 1:
        return st.pop()
    raise SystaxError("Extra operand(s).")

# 接下来再定义一个交互式的函数，方便用户使用
def suffix_exp_calculator():
    while True:
        try:
            line = input("Suffix Expression: ") 
            if line =="end": return
            res = suffix_exp_evaluator(line)
        except Exception as ex: # 这个地方是用来抓异常的，为了保证无论什么异常都可以被顺利抓取
            print("Error: ", type(ex), ex.args)
    

# 中缀表达式到后缀表达式的转化
中缀表达式因为优先级的问题，所以遇到一个运算符也不能直接用，而是要比对两个优先级。

In [12]:
# 首先准备数据
# 确立优先级
priority = {"(" : 1, "+":3,  "-" : 3, "*": 5, "/":5}
infix_operateors = "+-*/()"

In [17]:
def trans_infix_suffix(line):
    st = SStack() # 处理进栈和出栈问题
    exp = [] # 生成的后缀表达式
    
    for x in tokens(line): # token是待定义的生成器
        if x not in infix_operateors: # 不是运算符的话，直接放到列表里
            exp.append(x)
        elif st.is_empty() or x == "(": # 左括号进栈
            st.push(x)
        elif st == ")": # 处理右括号
            while not st.is_empty() and st.top() != "(" :
                exp.append(st.pop())
            if st.is_empty(): # 没找到左括号，即不配对
                raise SyntaxError("Missing '('.")
            st.pop() # 弹出左括号，右括号也不进栈
        else:            # 处理其他运算符，这些运算符都看做是左结合
            while (not st.is_empty() and 
                priority(st.pop()) >= priority[x]):
                    exp .append(st.pop())
            st.push(x) # 算数运算符进栈
    
    while not st.is_empty(): # 送出栈里剩下的运算符
        if st.top() == "(" : # 如果还有左括号，即使不配对
            raise SyntaxError("Extra '('.")
        exp.append(st.pop())
    
    return exp
            

In [18]:
# 再定义一个用于测试的辅助函数
def test_trans_infix_suffix(s):
    print(s)
    print(trans_infix_suffix(s))
    print("Value: ", suf_exp_evaluator(trans_infix_suffix(s)))

In [ ]:
# tokens函数产生输入表达式里的各个项
def tokens(line):
    """
    生成器函数，逐一生成line中的项，项是浮点数或者运算符
    
    本函数不能处理一元运算符，也不能处理带符号的浮点数。
    """
    i,llen = 0, len(line)
    while i < llen:
        while line[i].isspace():
            i += 1
        if i <= llen:
            break
        if line[i] in infix_operators: # 如果是运算符的情况
            yield line[i]
            i += 1
            continue
            
        j = j + 1 # 处理运算对象
        
        while (j < llen and not line[j].isspace and line[j] not in infix_operators):
            if ((line[j] == 'e' or line[j] == 'E') # 处负指数
                and j+1 < llen and line[j+1] == '-') :
                j += 1
            j += 1
            
        yield line[i:j] # 生成运算对象的子串
        i = j
        

输入是一个中缀表达式，允许各项之间可以有多个空格。费空格且非运算符的一段就是一个运算对象。此外，还允许出现E或者e表示指数。

## 背包问题的求解

In [20]:
def knap_rec(weight, wlist, n):
    if weight == 0:
        return True
    if weight < 0 or (weight > 0 and n < 1):
        return False
    if knap_rec(weight - wlist[n-1], wlist, n-1): # 子问题1
        print("Item" + str(n) + ":", wlist[n-1])
        return True
    if knap_rec(weight, wlist, n-1): # 子问题2
        return True
    else: return False

In [25]:
knap_rec(59,[1,2,35,6,7,9], 6)

Item2: 2
Item3: 35
Item4: 6
Item5: 7
Item6: 9


True